In [ ]:
!pip install kagglehub


In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sooyoungher/smoking-drinking-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/smoking-drinking-dataset


In [ ]:
import os
import pandas as pd

# Build full path to CSV file
csv_path = os.path.join(path, "smoking_driking_dataset_Ver01.csv")  # adjust filename if needed

# Load into DataFrame
drinking_df = pd.read_csv(csv_path)

# Check data
drinking_df.head()


,sex,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd,DRK_YN
0,Male,35,170,75,90.0,1.0,1.0,1.0,1.0,120.0,...,126.0,92.0,17.1,1.0,1.0,21.0,35.0,40.0,1.0,Y
1,Male,30,180,80,89.0,0.9,1.2,1.0,1.0,130.0,...,148.0,121.0,15.8,1.0,0.9,20.0,36.0,27.0,3.0,N
2,Male,40,165,75,91.0,1.2,1.5,1.0,1.0,120.0,...,74.0,104.0,15.8,1.0,0.9,47.0,32.0,68.0,1.0,N
3,Male,50,175,80,91.0,1.5,1.2,1.0,1.0,145.0,...,104.0,106.0,17.6,1.0,1.1,29.0,34.0,18.0,1.0,N
4,Male,50,165,60,80.0,1.0,1.2,1.0,1.0,138.0,...,117.0,104.0,13.8,1.0,0.8,19.0,12.0,25.0,1.0,N


In [ ]:
any_null = drinking_df.isna().sum().sum()
print("Any missing values:", any_null)

Any missing values: 0


In [ ]:

print(drinking_df.info())
print(drinking_df.describe())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 991346 entries, 0 to 991345
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   sex               991346 non-null  object 
 1   age               991346 non-null  int64  
 2   height            991346 non-null  int64  
 3   weight            991346 non-null  int64  
 4   waistline         991346 non-null  float64
 5   sight_left        991346 non-null  float64
 6   sight_right       991346 non-null  float64
 7   hear_left         991346 non-null  float64
 8   hear_right        991346 non-null  float64
 9   SBP               991346 non-null  float64
 10  DBP               991346 non-null  float64
 11  BLDS              991346 non-null  float64
 12  tot_chole         991346 non-null  float64
 13  HDL_chole         991346 non-null  float64
 14  LDL_chole         991346 non-null  float64
 15  triglyceride      991346 non-null  float64
 16  hemoglobin        99

In [ ]:
drinking_df_with_duplicates = drinking_df.shape[0]
drinking_df = drinking_df.drop_duplicates()
drinking_df_without_duplicates = drinking_df.shape[0]

print(
    f"We dropped a total of {drinking_df_with_duplicates-drinking_df_without_duplicates} entries. \nTherefore we keep {(drinking_df_without_duplicates/drinking_df_with_duplicates)*100}% of our original data."
)

We dropped a total of 26 entries. 
Therefore we keep 99.99737730318174% of our original data.


In [ ]:
metric_cols = [
    "age",
    "height",
    "weight",
    "waistline",
    "sight_left",
    "sight_right",
    "SBP",
    "DBP",
    "BLDS",
    "tot_chole",
    "HDL_chole",
    "LDL_chole",
    "triglyceride",
    "hemoglobin",
    "serum_creatinine",
    "SGOT_AST",
    "SGOT_ALT",
    "gamma_GTP",
]
ordinal_cols = ["hear_left", "hear_right", "urine_protein", "SMK_stat_type_cd"]
nominal_cols = ["sex", "DRK_YN"]

print(f"Metric Columns: \n{metric_cols}")
print(f"\nOrdinal Columns: \n{ordinal_cols}")
print(f"\nNominal Columns: \n{nominal_cols}")

Metric Columns: 
['age', 'height', 'weight', 'waistline', 'sight_left', 'sight_right', 'SBP', 'DBP', 'BLDS', 'tot_chole', 'HDL_chole', 'LDL_chole', 'triglyceride', 'hemoglobin', 'serum_creatinine', 'SGOT_AST', 'SGOT_ALT', 'gamma_GTP']

Ordinal Columns: 
['hear_left', 'hear_right', 'urine_protein', 'SMK_stat_type_cd']

Nominal Columns: 
['sex', 'DRK_YN']


In [ ]:
smoking_df_before_outlierD = drinking_df.shape[0]

columns_to_check = [
    "age",
    "height",
    "weight",
    "waistline",
    "sight_left",
    "sight_right",
    "SBP",
    "DBP",
    "BLDS",
    "tot_chole",
    "HDL_chole",
    "LDL_chole",
    "triglyceride",
    "hemoglobin",
    "serum_creatinine",
    "SGOT_AST",
    "SGOT_ALT",
    "gamma_GTP",
]
z_scores = (
    drinking_df[columns_to_check] - drinking_df[columns_to_check].mean()
) / drinking_df[columns_to_check].std()

# 3σ Standart deviation
threshold = 3
outliers = np.abs(z_scores) > threshold
outlier_columns = outliers.columns[outliers.any()]
drinking_df_cleaned = drinking_df[~outliers.any(axis=1)]

smoking_df_after_outlierD = drinking_df_cleaned.shape[0]

print(
    f"Due to the removal of outliers the amout of entries reduced from {smoking_df_before_outlierD} to {smoking_df_after_outlierD} by {smoking_df_before_outlierD-smoking_df_after_outlierD} entries."
)

Due to the removal of outliers the amout of entries reduced from 991320 to 904029 by 87291 entries.


In [ ]:
smoking_df_before_outlierD = drinking_df.shape[0]

columns_to_check = [
    "age",
    "height",
    "weight",
    "waistline",
    "sight_left",
    "sight_right",
    "SBP",
    "DBP",
    "BLDS",
    "tot_chole",
    "HDL_chole",
    "LDL_chole",
    "triglyceride",
    "hemoglobin",
    "serum_creatinine",
    "SGOT_AST",
    "SGOT_ALT",
    "gamma_GTP",
]


z_scores = (
    drinking_df[columns_to_check] - drinking_df[columns_to_check].mean()
) / drinking_df[columns_to_check].std()

# 3σ Standart deviation
threshold = 3
outliers = np.abs(z_scores) > threshold
outlier_columns = outliers.columns[outliers.any()]
drinking_df_cleaned = drinking_df[~outliers.any(axis=1)]

smoking_df_after_outlierD = drinking_df_cleaned.shape[0]

print(
    f"Due to the removal of outliers the amout of entries reduced from {smoking_df_before_outlierD} to {smoking_df_after_outlierD} by {smoking_df_before_outlierD-smoking_df_after_outlierD} entries."
)

Due to the removal of outliers the amout of entries reduced from 991320 to 904029 by 87291 entries.


In [ ]:
df_x, df_y = (
    drinking_df_cleaned.drop("DRK_YN", axis=1),
    drinking_df_cleaned[["DRK_YN"]],
)

df_y["DRK_YN"] = df_y["DRK_YN"].replace({"Y": 1, "N": 0})



In [ ]:
cats = df_x.select_dtypes(exclude=np.number).columns.tolist()

for col in cats:
    df_x[col] = df_x[col].astype("category")


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_x, df_y, test_size=0.3, random_state=1, stratify=df_y, shuffle=True
)

In [ ]:
params_xgb = {
    "tree_method": "gpu_hist", # Changed from "cuda" to "gpu_hist"
    "n_estimators": 200,
    "learning_rate": 0.1,
    "max_depth": 5,
    "min_child_weight": 1,
    "subsample": 1.0,
    "colsample_bytree": 1.0,
    "gamma": 2,
    "reg_lambda": 1,
    "objective": "binary:logistic",
    "eval_metric": "logloss",
    "random_state": 0,
}

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape)
print(X_train.isnull().sum().sum(), X_test.isnull().sum().sum())


(632820, 23) (632820, 1)
(271209, 23)
0 0


In [ ]:
import xgboost as xgb
import time

# Ensure target is 1D numeric
y_train = y_train.values.ravel().astype(int)
y_test = y_test.values.ravel().astype(int)

# Convert categorical features to numeric codes
for col in X_train.select_dtypes(include='category').columns:
    X_train[col] = X_train[col].cat.codes
    X_test[col] = X_test[col].cat.codes

# Ensure DataFrame is numeric
X_train = X_train.apply(pd.to_numeric)
X_test = X_test.apply(pd.to_numeric)

# Train without enable_categorical
params_xgb["tree_method"] = "hist"  # Use CPU-safe method
clf = xgb.XGBClassifier(**params_xgb)

start_time_xgb = time.time()
clf.fit(X_train, y_train)
end_time_xgb = time.time()

print("Training time:", end_time_xgb - start_time_xgb)

# Predictions
predictions_xgb = clf.predict(X_test)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Accuracy
accuracy = accuracy_score(y_test, predictions_xgb)
print("Accuracy:", accuracy)

# Classification report (precision, recall, f1-score)
print("\nClassification Report:\n", classification_report(y_test, predictions_xgb))

# Confusion Matrix
print("\nConfusion Matrix:\n", confusion_matrix(y_test, predictions_xgb))


print("F1 Score: ", f1_xgb)


Accuracy: 0.735167343266632

Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.74      0.74    137625
           1       0.73      0.74      0.73    133584

    accuracy                           0.74    271209
   macro avg       0.74      0.74      0.74    271209
weighted avg       0.74      0.74      0.74    271209


Confusion Matrix:
 [[101184  36441]
 [ 35384  98200]]
F1 Score:  0.7322210830459502


In [ ]:
!pip install optuna plotly


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 12.3 MB/s eta 0:00:00
